In [1]:
import pandas as pd
from dotenv import load_dotenv
import requests
import os
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver

# Tide-up the original dataframe

In [2]:
fifa_df_2 = pd.read_csv('csv/players_19.csv')
fifa_df_2 = fifa_df_2[fifa_df_2['overall'] >= 85]

In [3]:
fifa_df_2.head(3)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,31,1987-06-24,170,72,Argentina,FC Barcelona,...,63+2,61+2,61+2,61+2,63+2,58+2,47+2,47+2,47+2,58+2
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,26,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3


In [4]:
fifa_df_2.info(max_cols=105)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 104 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sofifa_id                   111 non-null    int64  
 1   player_url                  111 non-null    object 
 2   short_name                  111 non-null    object 
 3   long_name                   111 non-null    object 
 4   age                         111 non-null    int64  
 5   dob                         111 non-null    object 
 6   height_cm                   111 non-null    int64  
 7   weight_kg                   111 non-null    int64  
 8   nationality                 111 non-null    object 
 9   club                        111 non-null    object 
 10  overall                     111 non-null    int64  
 11  potential                   111 non-null    int64  
 12  value_eur                   111 non-null    int64  
 13  wage_eur                    111 no

In [5]:
fifa_19_pdata = fifa_df_2.iloc[:,[0,2,3,4,8,9,24]]
fifa_19_pstats = fifa_df_2.iloc[:, np.r_[0,10,11, 31:43]]

In [6]:
fifa_19_pdata.head()

,sofifa_id,short_name,long_name,age,nationality,club,team_position
0,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,Portugal,Juventus,LW
1,158023,L. Messi,Lionel Andrés Messi Cuccittini,31,Argentina,FC Barcelona,RW
2,190871,Neymar Jr,Neymar da Silva Santos Junior,26,Brazil,Paris Saint-Germain,CAM
3,193080,De Gea,David De Gea Quintana,27,Spain,Manchester United,GK
4,192985,K. De Bruyne,Kevin De Bruyne,27,Belgium,Manchester City,RCM


In [7]:
fifa_19_pstats.head()

,sofifa_id,overall,potential,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning
0,20801,94,94,90.0,93.0,81.0,89.0,35.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN
1,158023,94,94,88.0,91.0,88.0,96.0,32.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN
2,190871,92,93,92.0,84.0,83.0,95.0,32.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN
3,193080,91,93,NaN,NaN,NaN,NaN,NaN,NaN,90.0,85.0,87.0,94.0,58.0,88.0
4,192985,91,92,77.0,86.0,92.0,87.0,60.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN


# Get players links from fbref.com

In [8]:
player_link_df = fifa_19_pdata.iloc[:,[0,1,2]]
player_link_df['short_name'] = player_link_df['short_name'].str.lower().str.replace('.', '').str.replace(' ', '+')
player_link_df['long_name'] = player_link_df['long_name'].str.lower().str.replace(' ', '+').str.normalize('NFC')

In [9]:
player_link_df.head()

,sofifa_id,short_name,long_name
0,20801,cristiano+ronaldo,cristiano+ronaldo+dos+santos+aveiro
1,158023,l+messi,lionel+andrés+messi+cuccittini
2,190871,neymar+jr,neymar+da+silva+santos+junior
3,193080,de+gea,david+de+gea+quintana
4,192985,k+de+bruyne,kevin+de+bruyne


In [10]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
browser = webdriver.Chrome('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/pipelines-project/chromedriver', options=op)

In [11]:
def linkifyer(search_str, browser):
    browser.get("https://fbref.com/en/search/search.fcgi?search=" + search_str) 
    try:
        element = browser.find_element_by_class_name('filter')
        selector = browser.execute_script("return arguments[0].innerHTML;", element)
        link = BeautifulSoup(selector, 'html.parser').find('a')['href']
        return 'https://fbref.com'+link
    except:
        return 'javascript:void(0)'

In [12]:
column = []
for name in player_link_df['long_name']:
    column.append(linkifyer(name, browser))

KeyboardInterrupt: 

In [13]:
player_link_df['link'] = column

In [14]:
player_link_df['link'].value_counts()

javascript:void(0)                                                                             9
https://fbref.com/en/players/ecada4fc/all_comps/Keylor-Navas-Stats---All-Competitions          1
https://fbref.com/en/players/dea698d9/all_comps/Cristiano-Ronaldo-Stats---All-Competitions     1
https://fbref.com/en/players/3eed9821/all_comps/Samuel-Umtiti-Stats---All-Competitions         1
https://fbref.com/en/players/b9fbae28/all_comps/NGolo-Kante-Stats---All-Competitions           1
                                                                                              ..
https://fbref.com/en/players/08511d65/all_comps/Sergio-Ramos-Stats---All-Competitions          1
https://fbref.com/en/players/7f3b388c/all_comps/Fabinho-Stats---All-Competitions               1
https://fbref.com/en/players/4cde5509/all_comps/Zlatan-Ibrahimovic-Stats---All-Competitions    1
https://fbref.com/en/players/49296448/all_comps/Joshua-Kimmich-Stats---All-Competitions        1
https://fbref.com/en/players/a

In [22]:
no_link = player_link_df[(player_link_df['link'] == 'javascript:void(0)') | (player_link_df['link'] == 'https://fbref.comjavascript:void(0)')]
no_link

,sofifa_id,short_name,long_name,link
6,176580,l+suárez,luis+alberto+suárez+díaz,javascript:void(0)
15,211110,p+dybala,paulo+bruno+exequiel+dybala,javascript:void(0)
28,200145,casemiro,carlos+henrique+venancio+casimiro,javascript:void(0)
38,176676,marcelo,marcelo+vieira+da+silva+júnior,javascript:void(0)
39,189511,sergio+busquets,sergio+busquets+i+burgos,javascript:void(0)
58,210257,ederson,ederson+santana+de+moraes,javascript:void(0)
73,189596,t+müller,thomas+müller,https://fbref.comjavascript:void(0)
81,212831,alisson,alisson+ramses+becker,javascript:void(0)
96,156616,f+ribéry,franck+bilal+ribéry,javascript:void(0)
107,187961,paulinho,josé+paulo+bezzera+maciel+júnior,javascript:void(0)


In [24]:
no_link_idx = no_link.index.tolist()
player_link_df.drop(index=no_link_idx, inplace=True)

In [25]:
column_r = []
for name in no_link['short_name']:
    column_r.append(linkifyer(name, browser))

In [26]:
no_link['link'] = column_r

In [27]:
no_link

,sofifa_id,short_name,long_name,link
6,176580,l+suárez,luis+alberto+suárez+díaz,https://fbref.comjavascript:void(0)
15,211110,p+dybala,paulo+bruno+exequiel+dybala,https://fbref.com/en/players/e0921a4f/all_comp...
28,200145,casemiro,carlos+henrique+venancio+casimiro,https://fbref.comjavascript:void(0)
38,176676,marcelo,marcelo+vieira+da+silva+júnior,https://fbref.comjavascript:void(0)
39,189511,sergio+busquets,sergio+busquets+i+burgos,https://fbref.com/en/players/5ab0ea87/all_comp...
58,210257,ederson,ederson+santana+de+moraes,https://fbref.comjavascript:void(0)
73,189596,t+müller,thomas+müller,https://fbref.comjavascript:void(0)
81,212831,alisson,alisson+ramses+becker,https://fbref.comjavascript:void(0)
96,156616,f+ribéry,franck+bilal+ribéry,https://fbref.com/en/players/f61c7c4e/all_comp...
107,187961,paulinho,josé+paulo+bezzera+maciel+júnior,https://fbref.comjavascript:void(0)


In [28]:
no_link_idx = no_link.index[no_link['link'] == 'https://fbref.comjavascript:void(0)'].tolist()

rest_links = ['https://fbref.com/en/players/a6154613/all_comps/Luis-Suarez-Stats---All-Competitions',
              'https://fbref.com/en/players/4d224fe8/all_comps/Casemiro-Stats---All-Competitions',
              'https://fbref.com/en/players/603116a7/all_comps/Marcelo-Stats---All-Competitions',
              'https://fbref.com/en/players/3bb7b8b4/all_comps/Ederson-Stats---All-Competitions',
              'https://fbref.com/en/players/3c6089ab/all_comps/Thomas-Muller-Stats---All-Competitions',
              'https://fbref.com/en/players/7a2e46a8/all_comps/Alisson-Stats---All-Competitions',
              'https://fbref.com/en/players/bb7337cc/all_comps/Paulinho-Stats---All-Competitions']

idx_links = list(zip(no_link_idx, rest_links))

In [29]:
for new_link in idx_links:
    no_link.at[new_link[0], 'link'] = new_link[1]

no_link

,sofifa_id,short_name,long_name,link
6,176580,l+suárez,luis+alberto+suárez+díaz,https://fbref.com/en/players/a6154613/all_comp...
15,211110,p+dybala,paulo+bruno+exequiel+dybala,https://fbref.com/en/players/e0921a4f/all_comp...
28,200145,casemiro,carlos+henrique+venancio+casimiro,https://fbref.com/en/players/4d224fe8/all_comp...
38,176676,marcelo,marcelo+vieira+da+silva+júnior,https://fbref.com/en/players/603116a7/all_comp...
39,189511,sergio+busquets,sergio+busquets+i+burgos,https://fbref.com/en/players/5ab0ea87/all_comp...
58,210257,ederson,ederson+santana+de+moraes,https://fbref.com/en/players/3bb7b8b4/all_comp...
73,189596,t+müller,thomas+müller,https://fbref.com/en/players/3c6089ab/all_comp...
81,212831,alisson,alisson+ramses+becker,https://fbref.com/en/players/7a2e46a8/all_comp...
96,156616,f+ribéry,franck+bilal+ribéry,https://fbref.com/en/players/f61c7c4e/all_comp...
107,187961,paulinho,josé+paulo+bezzera+maciel+júnior,https://fbref.com/en/players/bb7337cc/all_comp...


In [30]:
player_link_df_final = player_link_df.append(no_link, sort=True)
player_link_df_final.sort_index(inplace=True)

In [31]:
player_link_df_final['link'].value_counts()

https://fbref.com/en/players/ecada4fc/all_comps/Keylor-Navas-Stats---All-Competitions          1
https://fbref.com/en/players/4431aed2/all_comps/Ciro-Immobile-Stats---All-Competitions         1
https://fbref.com/en/players/c906ce0f/all_comps/Pepe-Stats---All-Competitions                  1
https://fbref.com/en/players/8778c910/all_comps/Manuel-Neuer-Stats---All-Competitions          1
https://fbref.com/en/players/2f557579/all_comps/Lorenzo-Insigne-Stats---All-Competitions       1
                                                                                              ..
https://fbref.com/en/players/7f3b388c/all_comps/Fabinho-Stats---All-Competitions               1
https://fbref.com/en/players/4cde5509/all_comps/Zlatan-Ibrahimovic-Stats---All-Competitions    1
https://fbref.com/en/players/c0cddebe/all_comps/Blaise-Matuidi-Stats---All-Competitions        1
https://fbref.com/en/players/49296448/all_comps/Joshua-Kimmich-Stats---All-Competitions        1
https://fbref.com/en/players/b

In [9]:
player_link_df_final[['sofifa_id', 'link']].to_csv('csv/players_link.csv', index=False)
fifa_19_pdata.to_csv('csv/players_data.csv', index=False)
fifa_19_pstats.to_csv('csv/players_stats.csv', index=False)